In [14]:
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.time import Time
from astropy import units as u
import time
from astropy.coordinates.angles import Angle
from astroquery.vizier import Vizier
from astropy.utils.metadata import MergeConflictWarning
import warnings

def get_catalog(name):
    return Vizier(columns=["*", "+_r"], row_limit=-1).get_catalogs(name)[0]

def searchDR3(skycoordinate: SkyCoord, radius: Angle) -> Table:
    start_time = time.time()
    print(f"Searching in Gaia DR3 I/355/gaiadr3 {radius} around {skycoordinate.ra, skycoordinate.dec, skycoordinate.distance}")
    filters = {'Gmag': '<17', 'Plx': '>0.3'}
    stars_fromDR3 = Vizier(columns=["*", "+_r"], row_limit=-1).query_region(
        skycoordinate, 
        radius=radius, 
        catalog="I/355/gaiadr3",
        column_filters=filters
    )[0]  
    stars_fromDR3['SkyCoord1'] = SkyCoord(
        ra=stars_fromDR3['RA_ICRS'],
        dec=stars_fromDR3['DE_ICRS'],
        distance=(stars_fromDR3['Plx']).to(u.pc, equivalencies=u.parallax()),
        pm_ra_cosdec=stars_fromDR3['pmRA'],
        pm_dec=stars_fromDR3['pmDE'],
        obstime=(Time('J2000') + 1 * u.Myr)
    )
    end_time = time.time()
    print(f"found {len(stars_fromDR3):,} sources in {end_time - start_time:.2f} seconds")
    return stars_fromDR3

def searchDR3_dist(skycoordinate: SkyCoord, radius: Angle) -> Table:
    start_time = time.time()
    print(f"Searching in Gaia DR3 distances I/352/gedr3dis {radius} around {skycoordinate.ra, skycoordinate.dec, skycoordinate.distance}")
    stars_fromDR3_dist = Vizier(columns=["*", "+_r"], row_limit=-1).query_region(
        skycoordinate, 
        radius=radius, 
        catalog="I/352/gedr3dis"
    )[0]
    stars_fromDR3_dist['SkyCoord2'] = SkyCoord(
        ra=stars_fromDR3_dist['RA_ICRS'],
        dec=stars_fromDR3_dist['DE_ICRS'],
        distance=(stars_fromDR3_dist['rgeo']),
        obstime=(Time('J2000') + 1 * u.Myr)
    )
    end_time = time.time()
    print(f"found {len(stars_fromDR3_dist):,} sources in {end_time - start_time:.2f} seconds")
    # filtered_catalog = stars_fromDR3_dist[skycoordinate.separation_3d(stars_fromDR3_dist['SkyCoord2']) < skycoordinate.distance*config['Cluster']['distance_tolerance']]
    return stars_fromDR3_dist

def merge_gaia_tables(stars_fromDR3: Table, stars_fromDR3_dist: Table) -> Table:
    start_time = time.time()
    print("Starting merge of DR3 and distance catalog data")

    warnings.simplefilter('ignore', MergeConflictWarning)
    merged = join(stars_fromDR3, stars_fromDR3_dist, keys='Source', join_type='inner')

    # Order the columns
    merged = merged['RA_ICRS_1', 'DE_ICRS_1', 'e_RA_ICRS', 'e_DE_ICRS', '_r_1',
                                       'HIP', 'TYC2', 'Source', 'rgeo', 'Plx', 'e_Plx',
                                       'pmRA', 'pmDE', 'e_pmRA', 'e_pmDE',
                                       'RUWE', 'Teff', 'logg', 'Gmag', 'BP-RP', 'BPmag', 'RPmag', 'RV', 'e_RV',
                                       'b_rgeo', 'B_rgeo', 'FG', 'e_FG', 'FBP', 'e_FBP', 'FRP', 'e_FRP', 'RAVE5', 'RAVE6']

    # Adding row for uncertainties in Gmag and BPmag and RPmag
    # values for Gaia G, G_BP, G_RP zero point uncertainties
    #refer https://astronomy.stackexchange.com/questions/38371/how-can-i-calculate-the-uncertainties-in-magnitude-like-the-cds-does
    #refer https://www.cosmos.esa.int/web/gaia/edr3-passbands
    sigmaG_0 = 0.0027553202
    sigmaGBP_0 = 0.0027901700
    sigmaGRP_0 = 0.0037793818

    merged['e_Gmag'] = np.sqrt((-2.5 / np.log(10) * merged['e_FG'] / merged['FG'])**2 + sigmaG_0**2)
    merged['e_BPmag'] = np.sqrt((-2.5 / np.log(10) * merged['e_FBP'] / merged['FBP'])**2 + sigmaGBP_0**2)
    merged['e_RPmag'] = np.sqrt((-2.5 / np.log(10) * merged['e_FRP'] / merged['FRP'])**2 + sigmaGRP_0**2)
    merged['e_BP-RP'] = merged['e_BPmag'] + merged['e_RPmag']

    merged['SkyCoord'] = SkyCoord(
        ra=merged['RA_ICRS_1'],
        dec=merged['DE_ICRS_1'],
        distance=(merged['rgeo']),
        pm_ra_cosdec=merged['pmRA'],
        pm_dec=merged['pmDE'],
        obstime=(Time('J2000') + 1 * u.Myr)
    )

    end_time = time.time()
    print(f"{len(merged):,} sources found by merging in {end_time - start_time:.2f} seconds")
    return merged

def get_stars_in_region(self) -> Table:
    c = ClusterDias(self.name).skycoord
    t1 = searchDR3(c,self.search_arcmin)
    t2 = searchDR3_dist(c,self.search_arcmin)
    t3 = merge_gaia_tables(t1,t2)
    t3.sort('Gmag')
    return t3 

In [5]:
SkyCoord.from_name("SNR G166.0+04.3")

<SkyCoord (ICRS): (ra, dec) in deg
    (81.625, 42.9333333)>

In [29]:
green = get_catalog("VII/272/snrs")

In [28]:
get_catalog("VII/272/snrs")

SNR,RAJ2000,DEJ2000,Dmaj,Dmin,u_Dmin,type,l_S_1GHz_,S_1GHz_,u_S_1GHz_,alpha,u_alpha,Names,Simbad
,,,arcmin,arcmin,,,,Jy,,,,,
str11,str8,str6,float32,float32,str1,str2,str1,float32,str1,float32,str2,str26,str6
G000.0+00.0,17 45 44,-29 00,3.5,2.5,,S,,100.0,?,0.80,?,Sgr A East,Simbad
G000.3+00.0,17 46 15,-28 38,15.0,8.0,,S,,22.0,,0.60,,,Simbad
G000.9+00.1,17 47 21,-28 09,8.0,--,,C,,18.0,?,--,v,,Simbad
G001.0-00.1,17 48 30,-28 09,8.0,--,,S,,15.0,,0.60,?,,Simbad
G001.4-00.1,17 49 39,-27 46,10.0,--,,S,,2.0,?,--,?,,Simbad
G001.9+00.3,17 48 45,-27 10,1.5,--,,S,,0.6,,0.60,,,Simbad
G003.7-00.2,17 55 26,-25 50,14.0,11.0,,S,,2.3,,0.65,,,Simbad
G003.8+00.3,17 52 55,-25 28,18.0,--,,S?,,3.0,?,0.60,,,Simbad


In [31]:
coord = SkyCoord(ra=30,dec=2,unit=u.deg)
coord = SkyCoord.from_name("SNR G166.0+04.3")
radius = 1*u.deg
searchDR3(coord, radius)

Searching in Gaia DR3 I/355/gaiadr3 1.0 deg around (<Longitude 81.625 deg>, <Latitude 42.9333333 deg>, <Quantity 1.>)
found 14,334 sources in 1.55 seconds


_r,RA_ICRS,DE_ICRS,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,FBP,e_FBP,BPmag,FRP,e_FRP,RPmag,BP-RP,RV,e_RV,Vbroad,GRVSmag,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,Teff,logg,__Fe_H_,Dist,A0,HIP,PS1,SDSS13,SKYM2,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,_2MASS,RAVE6,RAJ2000,DEJ2000,SkyCoord1
,deg,deg,,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,,,mag,,,mag,mag,km / s,km / s,km / s,mag,,,,,,,,,,,,K,log(cm.s**-2),,pc,mag,,,,,,,,,,,,,deg,deg,"deg,deg,pc"
float64,float64,float64,int64,float64,float64,float64,float32,float64,float64,float32,float64,float32,float64,float64,float32,float64,float32,float32,float64,float32,float32,float64,float64,float64,float32,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,int32,int64,int64,int32,str12,str15,str19,int32,str10,str16,str17,str21,float64,float64,SkyCoord
0.008503,81.63654871254,42.93236286330,195633252072133248,0.0532,0.0341,0.4771,0.0632,4.583,0.714,0.077,-4.527,0.056,1.032,1707107.39292,1.32e+04,10.106715,5.038e+04,579.9,13.582963,2.812e+06,5.333e+04,8.625454,4.957509,--,--,--,--,0,0,0,1,1,0,1,0,0,1,0,3386.0,0.1079,0.7626,3735.0947,3.2623,--,--,--,--,,URAT1-665155321,,1018473,NC96000073,,05263276+4255565,,81.63654438102,42.93238298472,"81.63654871254,42.9323628633,2095.9966464053655"
0.010488,81.63455124943,42.94115149629,195633256371002496,0.0233,0.0156,0.6533,0.0267,8.263,2.938,0.034,-7.723,0.025,1.039,17735.86849,5.352,15.065236,8793,14.28,15.478221,1.299e+04,16.26,14.464232,1.013989,--,--,--,--,0,0,0,1,0,0,1,0,1,1,0,6407.0,4.2235,-1.0823,1179.2158,0.9304,--,159530816345170059,--,--,,URAT1-665155310,,1018471,NC96004812,,05263229+4256282,,81.63453341263,42.94118582027,"81.63455124943,42.94115149629,1530.6903413439459"
0.015768,81.60423028846,42.92918404038,195633290730748544,0.0192,0.0125,0.3354,0.0222,3.066,0.748,0.028,-2.973,0.021,1.028,26973.16616,5.26,14.610037,1.309e+04,20.05,15.046293,2.001e+04,17.12,13.994703,1.051590,-6.74,6.20,--,13.756598,0,0,0,1,1,0,0,0,0,1,0,6754.7,3.6982,-0.3178,2460.8284,1.2240,--,159510816041945707,--,--,,URAT1-665155199,J052625.01+425545.1,1018470,NCAD004342,,05262500+4255451,,81.60422575065,42.92919725574,"81.60423028846,42.92918404038,2981.514609421586"
0.019693,81.59811476243,42.93303518976,195633290730749056,0.0462,0.0299,0.5270,0.0536,7.623,4.065,0.066,-6.449,0.048,1.034,5174.69667,2.257,16.402655,2346,9.617,16.912762,4103,11.37,15.715029,1.197733,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,5729.7,4.6670,-0.9577,997.2222,0.9464,--,159520815980810312,--,--,,URAT1-665155177,J052623.53+425558.9,--,NCAD004364,,05262353+4255591,,81.59809008374,42.93306385166,"81.59811476243,42.93303518976,1897.533206831119"
0.020854,81.61260055545,42.95209988890,195645041761386112,0.0418,0.0272,0.3465,0.0463,0.633,0.097,0.059,-0.625,0.043,0.970,6047.80561,2.333,16.233372,2884,8.967,16.688612,4524,9.515,15.609165,1.079447,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,6328.1,4.4236,-0.7222,1570.6761,1.0347,--,159540816125813176,--,--,,URAT1-665155234,,--,NC96005059,,05262701+4257076,,81.61259996752,42.95210266811,"81.61260055545,42.9520998889,2886.002886002886"
0.021120,81.61419477478,42.91374869975,195633011554109696,0.0452,0.0286,1.9088,0.0489,12.692,-6.123,0.061,-11.117,0.044,1.017,6343.87073,2.717,16.181480,2132,10.26,17.016417,6087,10.02,15.286949,1.729467,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,4304.1,4.6669,0.0944,482.8600,0.6917,--,159490816141847177,--,--,,URAT1-665155239,,--,NC96004317,,05262741+4254496,,81.61423193510,42.91379810727,"81.61419477478,42.91374869975,523.8893545683151"
0.024166,81.59395892628,42.92513510590,195633187651536000,0.0430,0.0280,0.4068,0.0492,4.307,0.934,0.060,-4.204,0.045,0.970,5576.04424,2.312,16.321552,2587,12.79,16.806568,4293,11.08,15.665901,1.140667,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,6048.7,4.4900,-1.1509,1266.8905,1.0158,--,159510815939310827,--,--,,URAT1-665155159,J052622.53+425530.8,--,NCAD004304,,0